# Import and Install Dependencies

In [1]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib seaborn

# Getting Coordinates of left hand, right hand, arms and nose from Mediapipe

In [3]:
import mediapipe as mp
import cv2
from matplotlib import pyplot as plt

def extract_landmarks(frame, left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks):
    image_height, image_width, _ = frame.shape

    mp_drawing = mp.solutions.drawing_utils
    mp_holistic = mp.solutions.holistic

    with mp_holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image
        results = holistic.process(image_rgb)

        # Left hand landmarks
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            for landmark in results.left_hand_landmarks.landmark:
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)
                left_hand_landmarks.append((x, y))
                cv2.putText(frame, f"Left Hand: ({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

        # Right hand landmarks
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            for landmark in results.right_hand_landmarks.landmark:
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)
                right_hand_landmarks.append((x, y))
                cv2.putText(frame, f"Right Hand: ({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

        # Arm landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            arm_landmarks_indices = [mp.solutions.holistic.PoseLandmark.LEFT_WRIST,
                                     mp.solutions.holistic.PoseLandmark.RIGHT_WRIST,
                                     mp.solutions.holistic.PoseLandmark.LEFT_ELBOW,
                                     mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW,
                                     mp.solutions.holistic.PoseLandmark.LEFT_SHOULDER,
                                     mp.solutions.holistic.PoseLandmark.RIGHT_SHOULDER]
            for landmark_idx in arm_landmarks_indices:
                landmark = results.pose_landmarks.landmark[landmark_idx]
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)
                arm_landmarks.append((x, y))
                cv2.putText(frame, f"Arm: ({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)

        # Nose landmark
        if results.pose_landmarks:
            nose_landmark = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
            x = int(nose_landmark.x * image_width)
            y = int(nose_landmark.y * image_height)
            nose_landmarks.append((x, y))
            cv2.putText(frame, f"Nose: ({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)

    return frame

def process_landmarks(left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks):
    # Process the left-hand, right-hand, arm, and nose landmarks
    # Here, we simply print the coordinates
    print("Left Hand Landmarks:")
    for idx, landmark in enumerate(left_hand_landmarks):
        print(f"Landmark {idx}: {landmark}")
    print()
    print("Right Hand Landmarks:")
    for idx, landmark in enumerate(right_hand_landmarks):
        print(f"Landmark {idx}: {landmark}")
    print()
    print("Arm Landmarks:")
    for idx, landmark in enumerate(arm_landmarks):
        print(f"Landmark {idx}: {landmark}")
    print()
    print("Nose Landmarks:")
    for idx, landmark in enumerate(nose_landmarks):
        print(f"Landmark {idx}: {landmark}")
    print()

def show_landmarks_on_webcam():
    mp_holistic = mp.solutions.holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # Webcam parameters
    width, height = 640, 480

    # Open webcam
    cap = cv2.VideoCapture(1)
    cap.set(3, width)
    cap.set(4, height)

    left_hand_landmarks = []
    right_hand_landmarks = []
    arm_landmarks = []
    nose_landmarks = []

    while cap.isOpened():
        ret, frame = cap.read()
        # Rescale the frame by a factor of 0.5
        #frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
        if not ret:
            break

        frame = extract_landmarks(frame, left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks)

        process_landmarks(left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks)

        image_height, image_width, _ = frame.shape
        origin = (image_width // 2, image_height // 2)
        x_axis = (image_width, image_height // 2)
        y_axis = (image_width // 2, image_height)
        cv2.putText(frame, f"Origin: {origin}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame, f"X-axis: {x_axis}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.putText(frame, f"Y-axis: {y_axis}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imshow('Landmarks', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    return left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks

left_hand_landmarks, right_hand_landmarks, arm_landmarks, nose_landmarks = show_landmarks_on_webcam()


KeyboardInterrupt: 

In [4]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# Collect Keypoint Values for Training and Testing

## DataSet collection with data augmentation

In [18]:
import cv2
import os
import time
import numpy as np
import mediapipe as mp

# Create the dataset directory if it doesn't exist
dataset_dir = "Dataset"
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

# Map action labels to folder names
actions = {
    0: "Lock Person",
    1: "Unlock Person",
    2: "Go to Base",
    3: "Follow",
    4: "Stop",
    5: "Turn Left",
    6: "Turn Right",
    7: "Move Forward",
    8: "Move Backward"
}

# Set up MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Webcam parameters
#width, height = 640, 480

# Open webcam
cap = cv2.VideoCapture(0)
#cap.set(3, width)
#cap.set(4, height)

# Number of frames to collect for each action
num_frames_per_action = 60

# Number of repetitions for each action
num_repetitions = 30

# Data augmentation parameters
flip_prob = 0.5  # Probability of flipping the data
rotation_range = 20  # Range of rotation angle in degrees
scale_range = (0.8, 1.2)  # Range of scaling factors
noise_std = 0.05  # Standard deviation of Gaussian noise

for action_idx, action_label in actions.items():
    # Create the action's directory if it doesn't exist
    action_dir = os.path.join(dataset_dir, action_label)
    if not os.path.exists(action_dir):
        os.makedirs(action_dir)

    for repetition in range(num_repetitions):
        # Inform the start of data collection for each frame
        print(f"Collecting data for '{action_label}' - Repetition {repetition + 1}")

        # Pause for 5 seconds before starting data collection
        time.sleep(5)

        action_data = []

        for frame_num in range(num_frames_per_action):
            # Read frame from webcam
            ret, frame = cap.read()

            # Rescale the frame by a factor of 0.5
            frame = cv2.resize(frame, None, fx=0.5, fy=0.5)

            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame with MediaPipe Holistic
            with mp_holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5,
                                      min_tracking_confidence=0.5) as holistic:
                # Perform landmark detection
                results = holistic.process(frame_rgb)

                # Check if landmarks are detected
                if results.pose_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
                    landmark_coords = []

                    # Extract relevant landmarks (arms and nose)
                    arm_landmarks_indices = [
                        mp_holistic.PoseLandmark.LEFT_WRIST, mp_holistic.PoseLandmark.RIGHT_WRIST,
                        mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.RIGHT_ELBOW,
                        mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER
                    ]
                    nose_landmark_index = mp_holistic.PoseLandmark.NOSE.value

                    # Extract arm landmarks
                    for landmark_idx in arm_landmarks_indices:
                        landmark = results.pose_landmarks.landmark[landmark_idx]
                        landmark_coords.append([landmark.x, landmark.y, landmark.z])

                    # Extract nose landmark
                    nose_landmark = results.pose_landmarks.landmark[nose_landmark_index]
                    landmark_coords.append([nose_landmark.x, nose_landmark.y, nose_landmark.z])

                    # Extract left hand landmarks
                    for landmark in results.left_hand_landmarks.landmark:
                        landmark_coords.append([landmark.x, landmark.y, landmark.z])

                    # Extract right hand landmarks
                    for landmark in results.right_hand_landmarks.landmark:
                        landmark_coords.append([landmark.x, landmark.y, landmark.z])

                    # Append the landmark coordinates to the action's data
                    action_data.append(landmark_coords)

                    # Display landmarks and connections on the frame
                    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

                    # Draw action label, frame number, and repetition number on the frame
                    cv2.putText(frame, f"Action: {action_label}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                                cv2.LINE_AA)
                    cv2.putText(frame, f"Frame: {frame_num + 1}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                                cv2.LINE_AA)
                    cv2.putText(frame, f"Repetition: {repetition + 1}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1,
                                (0, 255, 0), 2, cv2.LINE_AA)

                    # Show the frame
                    cv2.imshow('Data Collection', frame)

            # Exit loop if 'q' is pressed
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('p'):  # Pause data collection when 'p' is pressed
                print("Data collection paused. Press 'p' to resume.")
                while True:
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('p'):  # Resume data collection when 'p' is pressed again
                        print("Resuming data collection.")
                        break

        # Convert the action's data to a numpy array
        action_data = np.array(action_data)

        # Save the action's data as a numpy array
        np.save(os.path.join(action_dir, f"{repetition}.npy"), action_data)

        # Apply data augmentation to generate additional data
        augmented_data = []

        for original_frame_data in action_data:
            augmented_frame_data = np.copy(original_frame_data)

            if np.random.uniform(0, 1) < flip_prob:
                # Flip horizontally
                augmented_frame_data[:, 0] = 1 - augmented_frame_data[:, 0]  # Flip x-coordinates

            # Rotate the frame
            angle = np.random.uniform(-rotation_range, rotation_range)
            rows, cols, _ = frame.shape
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            augmented_frame_data[:, :2] = cv2.transform(augmented_frame_data[:, :2].reshape(1, -1, 2), M).reshape(-1, 2)

            # Scale the frame
            scale_factor = np.random.uniform(scale_range[0], scale_range[1])
            augmented_frame_data[:, :2] *= scale_factor

            # Add Gaussian noise to the frame
            noise = np.random.normal(0, noise_std, augmented_frame_data[:, :2].shape)
            augmented_frame_data[:, :2] += noise

            augmented_data.append(augmented_frame_data)

        augmented_data = np.array(augmented_data)

        # Save the augmented data as an additional repetition
        np.save(os.path.join(action_dir, f"{repetition + num_repetitions}.npy"), augmented_data)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()

Data collection paused. Press 'p' to resume.
Resuming data collection.
Data collection paused. Press 'p' to resume.
Resuming data collection.
Data collection paused. Press 'p' to resume.
Resuming data collection.
Data collection paused. Press 'p' to resume.
Resuming data collection.
Data collection paused. Press 'p' to resume.
Resuming data collection.


In [17]:
cap.release()
cv2.destroyAllWindows()

# Build and Train LSTM Neural Network

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import time

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
dataset_dir = "Dataset"

# Get the list of actions from the dataset directory
actions = sorted(os.listdir(dataset_dir))

# Set the number of repetitions per action
num_repetitions = 30

# Create empty lists to store the data and labels
data = []
labels = []

# Iterate over each action
for action_label in actions:
    action_dir = os.path.join(dataset_dir, action_label)
    # Iterate over each repetition of the action
    for repetition in range(num_repetitions):
        # Load the data for the current repetition
        action_data = np.load(os.path.join(action_dir, f"{repetition}.npy"))
        data.extend(action_data)
        labels.extend([action_label] * len(action_data))

# Convert the lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Normalize the data
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)

# One-hot encode the labels
num_classes = len(actions)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

# Train the model
start_time = time.time()
history = model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_split=0.2, callbacks=[early_stop])
end_time = time.time()
inference_time = end_time - start_time

# Save the trained LSTM model
model.save('lstm_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Plot accuracy and loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

# Evaluate confusion matrix
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true_labels, y_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=actions, yticklabels=actions)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Print inference time
print(f"Inference Time: {inference_time:.2f} seconds")

Epoch 1/2000
 38/308 [==>...........................] - ETA: 43s - loss: 2.1809 - accuracy: 0.1768

KeyboardInterrupt: 

The LSTM model in the provided code takes a sequence of 60 frames, where each frame represents the coordinates of various landmarks detected from the pose, left hand, and right hand. The shape of the input data is (60, 49, 3), where:

- 60 represents the number of frames in a sequence.
- 49 represents the number of landmarks and joints considered in each frame.
- 3 represents the x, y, and z coordinates of each landmark.

The model outputs a probability distribution over the different action classes. The shape of the output is (1, num_classes), where num_classes is the total number of action classes. The probability distribution is obtained using the softmax activation function, where each element in the output represents the probability of the corresponding action class.

During prediction, the frames are passed through the model in batches. The model predicts the action class for each frame in the sequence. The predicted action class is determined by finding the index with the highest probability in the output. The final recognized action is the one with the highest probability among the predicted action classes.

In summary, the input shape of the LSTM model is (60, 49, 3), and the output shape is (1, num_classes).

# Test in Real Time

In [6]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained LSTM model
model = load_model('lstm_model.h5')

# Map action labels to folder names
actions = {
    0: "Lock Person",
    1: "Unlock Person",
    2: "Go to Base",
    3: "Follow",
    4: "Stop",
    5: "Turn Left",
    6: "Turn Right",
    7: "Move Forward",
    8: "Move Backward"
}

# Set up MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Webcam parameters
#width, height = 640, 480

# Open webcam
cap = cv2.VideoCapture(1)
#cap.set(3, width)
#cap.set(4, height)

while True:
    # Read frame from webcam
    ret, frame = cap.read()
    
    # Rescale the frame by a factor of 0.5
    #frame = cv2.resize(frame, None, fx=0.5, fy=0.5)

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    with mp_holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        # Perform landmark detection
        results = holistic.process(frame_rgb)

        # Check if landmarks are detected
        if results.pose_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
            landmark_coords = []

            # Extract relevant landmarks (arms and nose)
            arm_landmarks_indices = [
                mp_holistic.PoseLandmark.LEFT_WRIST, mp_holistic.PoseLandmark.RIGHT_WRIST,
                mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.RIGHT_ELBOW,
                mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER
            ]
            nose_landmark_index = mp_holistic.PoseLandmark.NOSE.value

            # Extract arm landmarks
            for landmark_idx in arm_landmarks_indices:
                landmark = results.pose_landmarks.landmark[landmark_idx]
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Extract nose landmark
            nose_landmark = results.pose_landmarks.landmark[nose_landmark_index]
            landmark_coords.append([nose_landmark.x, nose_landmark.y, nose_landmark.z])

            # Extract left hand landmarks
            for landmark in results.left_hand_landmarks.landmark:
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Extract right hand landmarks
            for landmark in results.right_hand_landmarks.landmark:
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Convert the landmark coordinates to a numpy array
            landmark_coords = np.array([landmark_coords])

            # Make a prediction using the LSTM model
            prediction = model.predict(landmark_coords)
            action_index = np.argmax(prediction, axis=1)
            # Get the scalar value from the NumPy array
            action_index_scalar = action_index[0]  
            action_label = actions[action_index_scalar]
            action_prob = prediction[0][action_index_scalar]

            # Draw action label and probability on the frame
            cv2.putText(frame, f"Action: {action_label}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                        cv2.LINE_AA)
            cv2.putText(frame, f"Probability: {action_prob:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                        2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Gesture Recognition', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step


In [7]:
landmark_coords

array([[[ 8.15259337e-01,  7.67898023e-01, -6.31418407e-01],
        [ 4.13174868e-01,  8.34269404e-01, -1.20221210e+00],
        [ 7.18261540e-01,  1.02987123e+00, -1.32918081e-04],
        [ 1.03932500e-01,  1.01202607e+00, -6.33048296e-01],
        [ 5.65004349e-01,  7.20924973e-01,  2.40592554e-01],
        [ 2.17606500e-01,  7.25757658e-01, -1.41354427e-01],
        [ 4.72011626e-01,  4.60155666e-01, -4.28829849e-01],
        [ 8.02263081e-01,  7.38495827e-01, -6.32011421e-08],
        [ 7.87428677e-01,  6.85195088e-01, -1.01244003e-02],
        [ 7.95692444e-01,  6.25590622e-01, -1.14846285e-02],
        [ 8.16576838e-01,  5.88951111e-01, -1.41014270e-02],
        [ 8.32780182e-01,  5.60314059e-01, -1.57610383e-02],
        [ 8.44454944e-01,  6.03753209e-01,  9.34872124e-03],
        [ 8.64326954e-01,  5.69800556e-01,  2.90703471e-03],
        [ 8.74415338e-01,  5.48720539e-01, -4.63646790e-03],
        [ 8.82957757e-01,  5.30719757e-01, -1.06496951e-02],
        [ 8.63845527e-01

In [8]:
len(landmark_coords)

1

In [9]:
prediction

array([[1.9241806e-09, 1.9505488e-03, 6.9236339e-05, 1.0724920e-14,
        4.7798853e-12, 1.9300388e-21, 3.9995871e-10, 1.8334742e-15,
        9.9798030e-01]], dtype=float32)

In [10]:
action_index_scalar

8

In [11]:
action_label

'Move Backward'

In [12]:
action_prob

0.9979803

In [13]:
len(prediction)

1

In [15]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time

# Load the trained LSTM model
model = load_model('lstm_model.h5')

# Map action labels to folder names
actions = {
    0: "Lock Person",
    1: "Unlock Person",
    2: "Go to Base",
    3: "Follow",
    4: "Stop",
    5: "Turn Left",
    6: "Turn Right",
    7: "Move Forward",
    8: "Move Backward"
}

unknown_action_label = "Unknown Action"

# Set up MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Webcam parameters
# width, height = 640, 480

# Open webcam
cap = cv2.VideoCapture(1)
# cap.set(3, width)
# cap.set(4, height)

# Variables for collecting frames
frame_count = 0
frames = []

# Variables for action recognition
recognized_action = None
start_time = 0

# Variables for FPS calculation
prev_time = 0

# Define the positions for displaying text
fps_pos = (20, 40)
inference_pos = (20, 70)
probability_pos = (20, 100)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Rescale the frame by a factor of 0.5
    # frame = cv2.resize(frame, None, fx=0.5, fy=0.5)

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Holistic
    with mp_holistic.Holistic(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        # Perform landmark detection
        results = holistic.process(frame_rgb)

        # Check if landmarks are detected
        if results.pose_landmarks and results.left_hand_landmarks and results.right_hand_landmarks:
            landmark_coords = []

            # Extract relevant landmarks (arms and nose)
            arm_landmarks_indices = [
                mp_holistic.PoseLandmark.LEFT_WRIST, mp_holistic.PoseLandmark.RIGHT_WRIST,
                mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.RIGHT_ELBOW,
                mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER
            ]
            nose_landmark_index = mp_holistic.PoseLandmark.NOSE.value

            # Extract arm landmarks
            for landmark_idx in arm_landmarks_indices:
                landmark = results.pose_landmarks.landmark[landmark_idx]
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Extract nose landmark
            nose_landmark = results.pose_landmarks.landmark[nose_landmark_index]
            landmark_coords.append([nose_landmark.x, nose_landmark.y, nose_landmark.z])

            # Extract left hand landmarks
            for landmark in results.left_hand_landmarks.landmark:
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Extract right hand landmarks
            for landmark in results.right_hand_landmarks.landmark:
                landmark_coords.append([landmark.x, landmark.y, landmark.z])

            # Collect frames
            frames.append(landmark_coords)
            frame_count += 1
            
            # Convert the landmark coordinates to a numpy array
            landmark_coords = np.array([landmark_coords])

            # Make a prediction using the LSTM model
            prediction = model.predict(landmark_coords)
            action_index = np.argmax(prediction, axis=1)
            # Get the scalar value from the NumPy array
            action_index_scalar = action_index[0]  
            action_label = actions[action_index_scalar]
            action_prob = prediction[0][action_index_scalar]
            
            # When 60 frames are collected, make a prediction
            if frame_count == 60:

                # Convert the frames to a numpy array
                frames = np.array(frames)

                # Normalize the data
                frames = frames / np.max(frames)

                # Reshape the frames array
                # frames = frames.reshape(60, 49, 3)

                # Perform prediction
                start_time = time.time()
                prediction = model.predict(frames)
                end_time = time.time()
                inference_time = end_time - start_time

                # Get the predicted action
                action_index = np.argmax(prediction)
                action_prob = prediction[0][action_index]
                action_prob_final = np.argmax(action_prob)

                # Check if the predicted action index is within the action labels dictionary
                if action_index in actions:
                    action_label = actions[action_index]
                else:
                    action_label = unknown_action_label

                # Check if a new action has been recognized
                if recognized_action != action_label:
                    recognized_action = action_label

                # Reset variables for collecting frames
                frame_count = 0
                frames = []

    # Calculate and display FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Draw FPS on the frame (top left corner)
    cv2.putText(frame, f"FPS: {fps:.2f}", fps_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    # Draw inference time on the frame
    cv2.putText(frame, f"Inference Time: {inference_time:.2f}s", inference_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (0, 255, 0), 2, cv2.LINE_AA)

    # Draw probability of recognized action on the frame
    cv2.putText(frame, f"Probability: {action_prob[0]:.2f}", probability_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (0, 255, 0), 2, cv2.LINE_AA)


    # Draw action labels on the frame (top right corner)
    y = 40
    for i, (action_id, action_name) in enumerate(actions.items()):
        text_color = (0, 255, 0) if action_name == recognized_action else (255, 255, 255)
        cv2.putText(frame, f"{action_name}", (width - 200, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2,
                    cv2.LINE_AA)
        y += 30

    # Check for lock and unlock gestures
    if recognized_action == "Lock Person":
        cv2.putText(frame, "Dynamic hand gesture recognition has started", (20, height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    elif recognized_action == "Unlock Person":
        cv2.putText(frame, "Dynamic hand gesture recognition has stopped", (20, height - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Draw pose landmarks
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                              connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    # Draw left hand landmarks
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                              connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2))

    # Draw right hand landmarks
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                              connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2))

    # Render the frame
    cv2.imshow('Action Recognition', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy all windows
cap.release()
cv2.destroyAllWindows()

NameError: name 'inference_time' is not defined

KeyError: 9.511863e-27

In [127]:
cap.release()
cv2.destroyAllWindows()